In [1]:
import os
import pandas as pd
import numpy as np
import re

In [2]:
# Read data into a dataframe (Master File)
df_master = pd.read_excel("")

In [3]:
# Folder Path
folder = ""

# Processed Files Folder
processed = ""

# Error Files Folder
error = ""

In [9]:
files = []
count = 1
for file in os.listdir(folder):
    file_path = os.path.join(folder, file)
    if os.path.isfile(file_path):
        
        # Extract TIN from file name
        TIN = re.search(r"\d{9}", file).group()
        
        #Create an array of File Paths
        files.append(file_path)
        
         # create an ExcelFile object
        excel_file = pd.ExcelFile(file_path)
        
        # get a list of sheet names
        sheet_names = excel_file.sheet_names
        inputs = next(filter(lambda x: 'input' in x.lower(), sheet_names), "None")      
        excel_file.close()
        
        try:
            # Read files into a dataframe
            df = pd.read_excel(file_path, sheet_name= inputs)
        except: 
            try:
                # Read files into a dataframe
                df = pd.read_excel(file_path, sheet_name= inputs)
            except:                
                print("ERROR READING FILE")
                print("Moving File to error folder......")
                os.rename(file_path, error + os.path.basename(file_path))
                print("Moved")
                print("NEXT.......")
                print("")
                count = count + 1
                continue
        finally:
            print("File read into dataframe")
            
        # Get column names
        df.columns = df.columns.str.lower()
        
        # Select columns of interest
        date = df.filter(like='date').columns
        tin = df.filter(like='tin').columns
        supplier = df.filter(like='supplier').columns
        invoice = df.filter(like='tax invoice').columns
        amount = df.filter(like='invoice amount').columns | df.filter(like='gross').columns
        vat = df.filter(like='vat').columns
        
        # Rename columns
        new_date = {col: col.replace(col, 'date') for col in date}
        new_tin = {col: col.replace(col, 'tin') for col in tin}
        new_supplier = {col: col.replace(col, 'supplier name') for col in supplier}
        new_invoice = {col: col.replace(col, 'tax invoice number') for col in invoice}
        new_amount = {col: col.replace(col, 'gross / invoice amount') for col in amount}
        new_vat = {col: col.replace(col, 'vat') for col in vat}
        
        df = df.rename(columns=new_date)
        #df = df.rename(columns=new_tin)
        df = df.rename(columns=new_supplier)
        df = df.rename(columns=new_invoice)
        df = df.rename(columns=new_amount)
        #df = df.rename(columns=new_vat)
        
        df.columns = df.columns.str.upper()
        #print(df.columns)
        '''
         # Rename colmns with wrong different naming convention
        if 'TAX INVOICE NUMBER/ REFERENCE' in df.columns:
            df = df.rename(columns={'TAX INVOICE NUMBER/ REFERENCE': 'TAX INVOICE NUMBER'})
        elif 'TAX INVOICE NUMBER / REFERENCE' in df.columns:
            df = df.rename(columns={'TAX INVOICE NUMBER / REFERENCE' : 'TAX INVOICE NUMBER'})
        elif 'TAX INVOICE NUMBER/REFERENCE' in df.columns:
            df = df.rename(columns={'TAX INVOICE NUMBER/REFERENCE' : 'TAX INVOICE NUMBER'})
        elif 'Tax Invoice Number' in df.columns:
            df = df.rename(columns={'Tax Invoice Number' : 'TAX INVOICE NUMBER'})
        '''
        
        # Count rows
        rows = len(df.index)
        
        # Create an array of TINs using the number of rows and put it in a pandas column
        print (f"Transforming File {count}........." )
        df["PRIMARY TIN"] = np.repeat(TIN, rows)        
        
        # Remove Empty Rows
        try:
            df_clean = df[df["DATE"].notnull() & df["SUPPLIER NAME"].notnull() & df["TAX INVOICE NUMBER"].notnull()]
        except:
            print("ERROR CLEANING DATAFRAME")
            print("Moving File to error folder......")
            os.rename(file_path, error + os.path.basename(file_path))
            print("Moved")
            print("NEXT.......")
            print("")
            count = count + 1
            continue
        else:         
            master_len = len(df_master.index)
            if (int(master_len) < 500000):
                #df_master = pd.concat([df_master, df_clean])
                df_master = pd.concat([df_master, df_clean], axis=0, ignore_index=True)
                #df_master = df_master.loc[:, ~df_master.columns.str.contains('^Unnamed')]
                
                # Move Transformed file to new folder
                os.rename(file_path, processed + os.path.basename(file_path))
                
                print("DONE")
                print("NEXT.........")
                print("")
            else:
                break;
            
            '''
            #Export File
            print (f"Writing File {count}........." )
            df.to_excel(file)
            count = count + 1
            '''
            count = count + 1

print("ALL FILES PROCESSED")



File read into dataframe
Index(['DATE', 'TIN', 'SUPPLIER NAME', 'TAX INVOICE NUMBER', 'VAT',
       'GROSS / INVOICE AMOUNT', 'PRIMARY TIN'],
      dtype='object')
Transforming File 1.........
DONE
NEXT.........

ERROR READING FILE
Moving File to error folder......
Moved
NEXT.......

File read into dataframe
File read into dataframe


C:\Users\vkhumalo\AppData\Local\Temp\ipykernel_7792\3126995025.py:40: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  amount = df.filter(like='invoice amount').columns | df.filter(like='gross').columns
C:\Users\vkhumalo\AppData\Local\Temp\ipykernel_7792\3126995025.py:40: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  amount = df.filter(like='invoice amount').columns | df.filter(like='gross').columns


Index(['DATE', 'TIN', 'CUSTOMER NAME', 'TAX INVOICE NUMBER', 'VAT',
       'GROSS / INVOICE AMOUNT'],
      dtype='object')
Transforming File 3.........
ERROR CLEANING DATAFRAME
Moving File to error folder......
Moved
NEXT.......



C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
C:\Users\vkhumalo\AppData\Local\Temp\ipykernel_7792\3126995025.py:40: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  amount = df.filter(like='invoice amount').columns | df.filter(like='gross').columns


File read into dataframe
Index(['ZERO RATED SUPPLIES - PURCHASES', 'UNNAMED: 1', 'UNNAMED: 2',
       'UNNAMED: 3', 'UNNAMED: 4', 'UNNAMED: 5', 'UNNAMED: 6', 'UNNAMED: 7',
       'UNNAMED: 8', 'UNNAMED: 9', 'UNNAMED: 10', 'UNNAMED: 11',
       'UNNAMED: 12'],
      dtype='object')
Transforming File 4.........
ERROR CLEANING DATAFRAME
Moving File to error folder......
Moved
NEXT.......



...

C:\Users\vkhumalo\AppData\Local\Temp\ipykernel_7792\3126995025.py:40: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  amount = df.filter(like='invoice amount').columns | df.filter(like='gross').columns


File read into dataframe
Index(['UNNAMED: 0', 'ZERO RATED SUPPLIES RECEIVED - PURCHASES', 'UNNAMED: 2',
       'UNNAMED: 3', 'UNNAMED: 4', 'UNNAMED: 5', 'UNNAMED: 6', 'UNNAMED: 7'],
      dtype='object')
Transforming File 29.........
ERROR CLEANING DATAFRAME
Moving File to error folder......
Moved
NEXT.......



C:\Users\vkhumalo\AppData\Local\Temp\ipykernel_7792\3126995025.py:40: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  amount = df.filter(like='invoice amount').columns | df.filter(like='gross').columns


File read into dataframe
Index(['DATE', 'TIN', 'CUSTOMER NAME', 'TAX INVOICE NUMBER',
       'GROSS / INVOICE AMOUNT'],
      dtype='object')
Transforming File 30.........
ERROR CLEANING DATAFRAME
Moving File to error folder......
Moved
NEXT.......

ALL FILES PROCESSED


C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\vkhumalo\AppData\Local\Temp\ipykernel_7792\3126995025.py:40: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  amount = df.filter(like='invoice amount').columns | df.filter(like='gross').columns


In [10]:
# Export master df to excel
print("Writing Master File......")
#df_master.to_excel("Master_4.xlsx")
df_master.to_csv("Master.csv")
print("FINISHED")

Writing Master File......
FINISHED
